In [13]:
import pandas as pd
import os
import psycopg2
import cx_Oracle
from sqlalchemy import create_engine, types

In [14]:
targetFolder = "../dataset2"

In [15]:
# os.listdir(폴더명): 폴더에 있는 파일 목록 리스트로 전환
fileList = os.listdir(targetFolder)
fileList

['CHEON.csv',
 'CHESUNGBUN.csv',
 'CHOLESTEROL.csv',
 'GOMILDO_CHOLESTEROL.csv',
 'HYULAP.csv',
 'HYULDANG.csv',
 'JEOMILDO_CHOLESTEROL.csv',
 'JUNGSUNGJIBANG.csv',
 'PEEUM.csv',
 'SANSOPOHWADO.csv',
 'SIMBAKSU.csv',
 'SIMJANGEUM.csv']

In [16]:
dbPrefix = "oracle+cx_oracle"
dbId = "yukyung"
dbPw = "yukyung"
dbIp = "192.168.110.111"
dbPort = "1521"
dbName = "orcl"

In [17]:
# engine: 저장할 DB 변수에 저장
engine = create_engine(f"{dbPrefix}://{dbId}:{dbPw}@{dbIp}:{dbPort}/{dbName}")

In [18]:
# for: 반복문으로 csv 파일 전체 호출한 후 DB 저장
for i in fileList:
    ## inDataset2: 파일 경로 변수에 저장
    inDataset2 = "../dataset2/{}".format(i)
    ## inDataset2Name: 파일명에 붙일 문자열 변수에 저장
    inDataset2Name = "_KIMYUKYUNG"
    
    ## if: 확장자명 '.csv'로 끝날 경우에만 파일 호출
    if i[-4:] == ".csv":
        ### indataName: 파일명(확장자명 제거한 파일명 + 파일명에 붙일 문자열) 변수에 저장
        indataName = i[:-4] + inDataset2Name
        ### encoding='UTF-8': 한글 인코딩
        indata = pd.read_csv(inDataset2, encoding='UTF-8')
    else:
        pass
    
    ## tableName: DB에 저장할 테이블명 변수에 저장
    ## .lower(): 데이터 이관시 대조(if_exists="replace")를 위해 적용 
    tableName = indataName.lower()  
    ## allColumns: 파일 컬럼 목록 리스트로 변수에 저장
    allColumns = list(indata.columns[:])

    ## typeDict: 파일 데이터 형변환을 위해 dictionary 생성
    typeDict={}
    floatNum = 'float64'
    intNum = 'int64'
    maxLen = 100

    ## for: 반복문으로 csv 파일 데이터 형변환
    ### types.NUMERIC: module 'sqlalchemy.types' has no attribute 'NUMBER'
    for column in allColumns:
        if indata[column].dtype == floatNum:
            typeDict[column] = types.FLOAT
        elif indata[column].dtype == intNum:
            typeDict[column] = types.NUMERIC
        else:
            typeDict[column] = types.VARCHAR(maxLen)

    indata.to_sql(name=tableName,
                  if_exists="replace",
                  con=engine,
                  dtype=typeDict,
                  index=False)